In [31]:
import numpy as np
import pandas as pd
import math
import operator
import random
import matplotlib.pyplot as plt
from numpy import linalg as LA
from copy import deepcopy
from sklearn import mixture
from sklearn.cluster import AgglomerativeClustering

In [4]:
def get_test_train(location):
    location = '~/Documents/SMAI/Assignment3/' + location
    dataframe = pd.read_csv(location,header=None)
    #dataframe = dataframe.sample(frac=1).reset_index(drop=True)
    train  = dataframe[:int(0.8*len(dataframe))]
    test = dataframe[int(0.8*len(dataframe)):]
    return test,train

In [5]:
def get_test_train_with_header(location):
    location = '~/Documents/SMAI/Assignment3/' + location
    dataframe = pd.read_csv(location)
    #dataframe = dataframe.sample(frac=1).reset_index(drop=True)
    train  = dataframe[:int(0.8*len(dataframe))]
    test = dataframe[int(0.8*len(dataframe)):]
    return dataframe,test,train

In [6]:
def getXY(location):
    location = '~/Documents/SMAI/Assignment3/' + location
    dataframe = pd.read_csv(location)
    X = dataframe.values[:,:-1]
    Y = dataframe.values[:,-1]
    return X,Y

In [7]:
def PCA(X):
    Z = np.array(X.copy())
    #Standardize
    for col in range(0,Z.shape[1]):
        Z.T[col] = (Z.T[col] - np.mean(Z.T[col]))/np.std(Z.T[col]) 
    #print(Z)
    #Find Covariance
    
    cov = np.cov(Z.T.astype(float))
    eigval,eigvect = LA.eig(cov)
    #print(eigval)
    #Sort eigen values
    eigvalvect = [[np.abs(eigval[i]),eigvect.T[i]] for i in range(0,eigval.shape[0])]
    #print(eigvalvect)
    eigvalvect.sort(key=lambda x:x[0],reverse=True)
    neweigvect = []
    for i in range (0,eigval.shape[0]):
        neweigvect.append(eigvalvect[i][1])
    neweigvect = np.array(neweigvect) #ordered eigen vector
    
    #Finding K with 10% error
    eigvalsum = sum(eigval)
    s=0
    k=0

    for i in range(0,eigval.shape[0]):
        s+=eigvalvect[i][0]/eigvalsum
        if s < 0.9:
            k+=1  
        #print(s)    
    #print(k+1)
    
    '''
    eignormlist = []
    Klist = []
    for i in range(0,eigval.shape[0]):
        eignormlist.append(eigvalvect[i][0]/eigvalsum)
        k+=1
        Klist.append(k)
    plt.plot(Klist,eignormlist,c='r')
    plt.show()
    '''
    #print(X.shape)
    #print(np.array(neweigvect[:k+1]).shape)
    projected_data = Z.dot(np.array(neweigvect[:k+1]).T)
    #print(neweigvect[:K+1])
    #print(projected_data)
    return(projected_data)

In [8]:
def KMeans(data,k):
    KPoints = {}
    Kval = {}
    KPointsRowNum = {}
    for i in range(0,k):
        KPoints[i] = []
        KPointsRowNum[i] = []
        Kval[i] = []
        
    #initialize clusters at random points (at some given point)
    for key in Kval:
        row = random.randint(0,data.shape[0])
        for j in data[row]:
            Kval[key].append(random.uniform(0,1))  
            #Kval[key].append(j)  #A random point in the dataset is the cluster's position
    print("C-1")
    #for each row
    #  find the distane to each cluster
    #  the closest cluster (least distance gets the row)
    
    #for each cluster 
    #  find the mean of the points in that cluster
    #  change the position of the cluster
    
    KPointsRowNumCopy = {}
    count=1
    #while count<=20:
    while KPointsRowNumCopy != KPointsRowNum:
        print(count)
        KPointsRowNumCopy = deepcopy(KPointsRowNum)
        for K in KPoints:
            KPoints[K] = []
            KPointsRowNum[K] = []
        #print("C0")
        
        for row in range(0,data.shape[0]):
            
            distlist = []
            for K in Kval:
                center = Kval[K]
                #data[row] and center
                dist = 0
                for attribute in range(0,len(data[row])):
                    dist+=(pow(abs(data[row][attribute] - center[attribute]),2))
                dist=math.sqrt(dist)
                distlist.append(dist)
            minclust = np.argmin(distlist)
            KPoints[minclust].append(data[row])
            KPointsRowNum[minclust].append(row)
            
        #print("C1")
        
        for K in KPoints:
            mat = np.array(KPoints[K]).T
            for i in range(0,len(Kval[K])):
                Kval[K][i] = np.sum(mat[i])/len(mat[i])
        #print("C2")
        
        count+=1
    return KPointsRowNum 
    

In [9]:
def findLabel(CalculatedCluster,NewData,Y):
    clusterLabelCount = {}
    clusterLabel = {}
    clusterDataSize = {}
    distinctLabels = np.unique(np.array(Y))
    for key in CalculatedCluster:
        #key is some cluster here.
        clusterLabelCount[key] = {}
        clusterLabel[key] = ''
        #clusterDataSize has the count of element in every cluster
        clusterDataSize[key] = len(CalculatedCluster[key])
        
        for lab in distinctLabels:
            #will have count of every label in a cluster.To be used to find ratio and assign a cluster name .
            clusterLabelCount[key][lab] = 0
    
    for key in CalculatedCluster:
        for row in CalculatedCluster[key]:
            clusterLabelCount[key][Y[row]]+=1
        clusterLabel[key] = sorted(clusterLabelCount[key].items(),key = lambda arg:arg[1],reverse=True)[0][0]
    
    #print(clusterLabel)  #Adds a label to a cluster
    #print(clusterLabelCount)  #Has the count of all data points in cluster
    
    
    return(clusterLabel,clusterLabelCount,clusterDataSize)
    


In [10]:
'''
USED IN Logistic REGRESSION
'''
def sigmoid(B,X):
    return 1/(1+np.exp(-1*(X.dot(B))))

In [11]:
'''
USED IN Logistic REGRESSION
'''
def gradient_descent(X,B,Y,alpha):
    m = len(Y)
    cost = 0
    for i in range(0,500):
        #print(B)
        B -= alpha*(1/m)*(X.T.dot((sigmoid(B,X) - Y)))/2
        #cost = cost_function(X,B,Y)
        #print(cost)
    return B

In [12]:
'''
USED IN Logistic REGRESSION
'''
def predictLinear(dataframe,test,target,B,multi=False):
    X = []
    x0 = np.ones(test.shape[0])
    columns = list(test.columns)
    columns.pop(0)
    X.append(list(x0))
    for col in columns:
        if col!= target:
            #print(col)
            normalized_col = list(((test[col].values) - np.mean(dataframe[col].values))/np.std(dataframe[col].values))
            X.append(normalized_col)
    
    X = np.array(X).T
    Y = np.array(test[target].values)
    if multi is True:
        predicted_y = sigmoid(B,X)
    else:
        predicted_y = sigmoid(B,X)>=0.5
    return X,predicted_y

In [13]:
'''
USED IN Logistic REGRESSION
'''
def LogisticRegressionFit(dataframe,test,train,target):
    #get X matrx which consists all features
    X = []
    x0 = np.ones(train.shape[0])
    columns = list(train.columns)
    columns.pop(0)
    X.append(list(x0))
    for col in columns:
        if col!= target:
            #print(col)
            normalized_col = list(((train[col].values) - np.mean(dataframe[col].values))/np.std(dataframe[col].values))
            X.append(normalized_col)
    
    X = np.array(X).T
    #X = np.array(X)
    B = np.array(np.zeros(len(columns)))
    Y = np.array(train[target].values)
    alpha = 0.1
    #B = Y.T.dot(np.linalg.inv(X.T.dot(X)).dot(X.T))
    #print(X.shape,B.shape,Y.shape)
    B = gradient_descent(X,B,Y,alpha)
    return(B)

In [14]:
def BinaryMeasure(predicted_y,test,target,multi=False):
    j = 0
    TN=0
    TP=0
    FN=0
    FP=0
    #print(test[target].values)
    if multi is True:
        target_col = (test[target].values)
        for i in predicted_y:
            if i == target_col[j]:
                TP+=1
            j+=1
        accuracy = TP/len(test[target])
        precision = 0
        recall = 0
    else:
        target_col = (test[target].values)>=0.5
        #print(target_col)
        #print(predicted_y,target_col)
        for i in predicted_y:
            #print(j,target)
            if i==0 and i == target_col[j]:
                TN+=1
            elif i==0 and i!= target_col[j]:
                FN+=1
            elif i==1 and i == target_col[j]:
                TP+=1
            elif i==1 and i!= target_col[j]:
                FP+=1
            j+=1
    
        #print(TP,TN,FP,FN)
        accuracy =  (TP+TN)/(TP+TN+FP+FN)
        precision = (TP)/(TP+FP+np.finfo(float).eps)
        recall = (TP)/(TP+FN+np.finfo(float).eps)
        #fscore = 2/((1/precision)+(1/recall))
    return accuracy,precision,recall

In [15]:
'''
USED IN KNN
'''
def minkowski_predict(test_row,train_row,power,target):
    dist = 0
    #print(test_row,train_row,power,target)
    for attribute in range(0,len(test_row)):
        if attribute != target:
            dist+=(pow(abs(test_row[attribute] - train_row[attribute]),power))
    return pow(dist,1/power)

In [16]:
'''
USED IN KNN
'''
def getsecond(item):
    return item[1]


def KNN(test,train,K,dist_measure,target):
    unique_values = np.unique(train.values[:,target])
    maxval = -1
    winner = None
    class_dict = {}
    for classVal in unique_values:
        class_dict[classVal] = 0
  
    if dist_measure == 'euclid':
        power = 2
    elif dist_measure == 'man':
        power = 1
    predicted = []
    for test_row in test.values:
        #print(test_row)
        knn = []
        maxval = -1
        winner = None
        for train_row in train.values:
            #print(train_row)
            #print(test_row)
            y = minkowski_predict(test_row,train_row,power,target)
            #print(y)
            knn.append((train_row[target],y))
        knn.sort(key = getsecond)
        
        #print(knn)
        for i in range(0,K):
            class_dict[knn[i][0]]+=1
            #if class_dict[knn[i][0]] > maxval:
            #    winner = knn[i][0]
            #    maxval = class_dict[knn[i][0]]
        winner = max(class_dict.items(),key=operator.itemgetter(1))[0]    
        predicted.append(winner)
        #print(class_dict)
        for classVal in unique_values:
            class_dict[classVal] = 0
    return predicted        

## Question 1
### Part-1

In [17]:
location = 'intrusion_detection/data.csv'
X,Y = getXY(location)
NewData = PCA(X)

### Part-2

In [176]:
#Continued from part-1
CalculatedCluster = KMeans(NewData,5)

#print(len(CalculatedCluster[0]))
#print(len(CalculatedCluster[1]))
#print(len(CalculatedCluster[2]))
#print(len(CalculatedCluster[3]))
#print(len(CalculatedCluster[4]))


C-1
1
2
3
4
5
6
7
8
9
10
11
12


In [177]:
clusterLabel,clusterLabelCount,clusterDataSize = findLabel(CalculatedCluster,NewData,Y)
print(clusterLabelCount)
for key in clusterLabel:
    print(clusterLabel[key])
    print(clusterLabelCount[key][clusterLabel[key]]/clusterDataSize[key])
    print("\n")

{0: {'dos': 0, 'normal': 8, 'probe': 0, 'r2l': 0, 'u2r': 0}, 1: {'dos': 931, 'normal': 12425, 'probe': 1076, 'r2l': 185, 'u2r': 10}, 2: {'dos': 6802, 'normal': 35, 'probe': 62, 'r2l': 0, 'u2r': 0}, 3: {'dos': 1366, 'normal': 540, 'probe': 547, 'r2l': 10, 'u2r': 0}, 4: {'dos': 15, 'normal': 356, 'probe': 628, 'r2l': 2, 'u2r': 0}}
normal
1.0


normal
0.8494564845833048


dos
0.9859399913030874


dos
0.5546082013804303


probe
0.6273726273726273




### Part-3

In [30]:
#PCA, X and Y are defined from PArt-1. Run Part-1 before running this
gmm = mixture.GaussianMixture(n_components=5)
gmm.fit(NewData)
labels = gmm.predict(NewData)
dictrowgmm = {}
for i in range(0,5):
    dictrowgmm[i] = []
for i in range(0,len(labels)):
    dictrowgmm[labels[i]].append(i)

clusterLabel,clusterLabelCount,clusterDataSize = findLabel(dictrowgmm,NewData,Y) 
#print(clusterLabelCount)
for key in clusterLabel:
    print(clusterLabel[key])
    print(clusterLabelCount[key][clusterLabel[key]]/clusterDataSize[key])
    print("\n")


normal
0.42648864333947206


normal
0.5222701149425287


dos
1.0


normal
1.0


normal
0.9323237103644108




### Part-4

In [36]:
#PCA, X and Y are defined from PArt-1. Run Part-1 before running this
clt = AgglomerativeClustering(linkage='complete', 
                              affinity='euclidean', 
                              n_clusters=5)
model = clt.fit(NewData)
labels = model.labels_
dictrowgmm = {}
for i in range(0,5):
    dictrowgmm[i] = []
for i in range(0,len(labels)):
    dictrowgmm[labels[i]].append(i)

clusterLabel,clusterLabelCount,clusterDataSize = findLabel(dictrowgmm,NewData,Y) 
#print(clusterLabelCount)
for key in clusterLabel:
    print(clusterLabel[key])
    print(clusterLabelCount[key][clusterLabel[key]]/clusterDataSize[key])
    print("\n")

normal
1.0


normal
0.8461538461538461


normal
0.5343156883158485


probe
1.0


normal
0.5




## Question -2

### Part-1

In [156]:
location = 'AdmissionDataset/data.csv'
dataframe,test,train = get_test_train_with_header(location)
target = 'Chance of Admit'
slopeValues = LogisticRegressionFit(test,train,target)
X,predicted_y = predictLinear(test,target,slopeValues)
accuracy,precision,recall = BinaryMeasure(predicted_y,test,target)
print(accuracy)

0.9555555555555556


### Part-2

In [157]:
location = 'AdmissionDataset/data.csv'
dataframe,test,train = get_test_train_with_header(location)
test = pd.DataFrame(np.delete(test.values,0,1))
train = pd.DataFrame(np.delete(train.values,0,1))
K = 7
target = 7
#print(test.values)
train.values.T[-1] = np.array(train.values.T[-1])>=0.5
test.values.T[-1] = np.array(test.values.T[-1])>=0.5
#print(test.values.T[-1])
predicted_y = KNN(test,train,K,'euclid',target)
#print(predicted_y)
#test.values[target] = np.array(test.values[target])>=0.5
accuracy,precision,recall = BinaryMeasure(np.array(predicted_y)>=0.5,test,target)
print("Robot1")
print("===================================")
print("Accuracy = ",accuracy)
print("Precision = ",precision)
print("Recall = ",recall)


Robot1
Accuracy =  0.9444444444444444
Precision =  0.9418604651162791
Recall =  1.0


### Part-3

## Question -3

### One-All

In [178]:
location = 'wine-quality/data.csv'
dataframe,test,train = get_test_train_with_header(location)
#print(test)
testcopy = test.copy()
traincopy = train.copy()
target = 'quality'
unique_class = np.unique(np.array(train[target]))
labpred = []
labdict = {}
true_predicted = []
z = 0
for key in unique_class:
    labdict[z] = key
    z+=1
print(labdict)
for c in unique_class:
    train[target] = (train[target].values) == c
    test[target] = (test[target].values) == c
    slopeValues = LogisticRegressionFit(dataframe,test,train,target)
    #print(slopeValues)
    X,predicted_y = predictLinear(dataframe,test,target,slopeValues,True)
    labpred.append(predicted_y)
    train = traincopy.copy()
    test = testcopy.copy()
    #print(predicted_y)
    #print(test[target].values)
labpred = np.array(labpred)
#print(labpred)
for row in range(0,len(testcopy[target])):
    true_predicted.append(labdict[(np.argmax(labpred[:,row]))])
#print(true_predicted)
#print(testcopy[target])
accuracy,precision,recall = BinaryMeasure(true_predicted,testcopy,target,True)
print(accuracy)

{0: 3, 1: 4, 2: 5, 3: 6, 4: 7, 5: 8, 6: 9}


/home/yash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.5374149659863946


### One-One

In [201]:
location = 'wine-quality/data.csv'
dataframe,test,train = get_test_train_with_header(location)
#print(test)
testcopy = test.copy()
traincopy = train.copy()
target = 'quality'
unique_class = np.unique(np.array(train[target]))
labpred = []
labdict = {}
true_predicted = []
z = 0
for key in unique_class:
    labdict[z] = key
    z+=1
    
for c1_ind in range(0,len(unique_class)):
    for c2_ind in range(c1_ind,len(unique_class)):
        if unique_class[c1_ind]!=unique_class[c2_ind]:
            train = train[(train[target] == unique_class[c1_ind]) | (train[target] == unique_class[c2_ind])]
            train = test[(test[target] == unique_class[c1_ind]) | (test[target] == unique_class[c2_ind])]
            #print(train)
            train[target] = (train[target].values) == unique_class[c1_ind]
            test[target] = (test[target].values) == unique_class[c1_ind]
            slopeValues = LogisticRegressionFit(dataframe,test,train,target)
            X,predicted_y = predictLinear(dataframe,test,target,slopeValues,True)
            for val in range(0,len(predicted_y)):
                if predicted_y[val] >= 0.5:
                    predicted_y[val] = unique_class[c1_ind]
                else:
                    predicted_y[val] = unique_class[c2_ind]
            labpred.append(predicted_y)
            train = traincopy.copy()
            test = testcopy.copy()
            #print(predicted_y)
            #print(test[target].values)
labpred = np.array(labpred)
predarr = []
for row in range(0,len(testcopy[target])):
    #print(row)
    col = []
    col.extend(labpred[:,row])
    count = max(col,key=col.count)
    true_predicted.append(count)
    #true_predicted.append(labdict[(np.argmax(labpred[:,row]))]) 
accuracy,precision,recall = BinaryMeasure(true_predicted,testcopy,target,True)
print(accuracy)

/home/yash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.5272108843537415
